In [1]:
!cp -r /kaggle/input/isic-2024-package/* .

In [2]:
# скачиваем необходимые библиотеки и модели
!mv ./setup/efficientnet_pytorch-0.7.1.tar.xyz ./setup/efficientnet_pytorch-0.7.1.tar.gz
!mv ./setup/pretrainedmodels-0.7.4.tar.xyz ./setup/pretrainedmodels-0.7.4.tar.gz
!pip install ./setup/efficientnet_pytorch-0.7.1.tar.gz
!pip install ./setup/munch-4.0.0-py2.py3-none-any.whl
!pip install ./setup/pretrainedmodels-0.7.4.tar.gz
!rm -rf ./setup

Processing ./setup/efficientnet_pytorch-0.7.1.tar.gz
  Preparing metadata (setup.py) ... - done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=7efda3bb67e969c0979bee073cf7319fc7a17a28c7ace801e1cd98860e2dfa24
  Stored in directory: /root/.cache/pip/wheels/05/b4/6c/b74f1eea0671c82226644bcbb0497d3e8cccd94febc031c466
Successfully built efficientnet_pytorch
Processing ./setup/munch-4.0.0-py2.py3-none-any.whl
Processing ./setup/pretrainedmodels-0.7.4.tar.gz
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=7819a565d4a536d3fb13cbe8c8839dba3ec82700997369bbd52977368a7d61b3
  Stored in directory: /root/.cache/pip/wheels/89/2c/e3/e433561365cb9cc3d3a221e0153f3f931fe7ebed27b2c4c4ca
Successfully built pretrainedmodels


In [3]:
# импортируем необходимые библиотеки
import os
import cv2
import h5py
import numpy as np
import pandas as pd
import pandas.api.types
from PIL import Image
from io import BytesIO
import polars as pl
from tqdm import tqdm
from scipy.stats import gmean
from typing import Optional, List

import sklearn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

import albumentations as albu
from albumentations.pytorch import ToTensorV2

import timm
from timm.models.layers import BatchNormAct2d, SelectAdaptivePool2d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import catboost as cb
import lightgbm as lgb
import xgboost as xgb

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_encoder
from segmentation_models_pytorch.decoders.unet.decoder import UnetDecoder
from segmentation_models_pytorch.decoders.unetplusplus.decoder import UnetPlusPlusDecoder
from segmentation_models_pytorch.decoders.fpn.decoder import FPNDecoder
from segmentation_models_pytorch.base import SegmentationHead
from segmentation_models_pytorch.base import initialization as init
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/albumentations/check_version.py:49: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()


In [4]:
# рпределение модели для классификации с использованием предобученной модели из библиотеки timm

class ISIC_Model(nn.Module):
    def __init__(self, model_name, pretrained, num_classes):
        super(ISIC_Model, self).__init__()
        self.backbone = timm.create_model(model_name, pretrained=pretrained, num_classes=num_classes)

    @autocast()
    def forward(self, x):
        return self.backbone(x)

#  модель Unet с предобученным энкодером EfficientNet и декодером для сегментации
class IsicSMPEffnetUnetModel(nn.Module):
    def __init__(
        self,
        encoder_name = "timm-efficientnet-b0",
        encoder_weights = 'noisy-student',
        decoder_use_batchnorm: bool = True,
        decoder_channels: List[int] = (256, 128, 64, 32, 16),
        decoder_attention_type: Optional[str] = None,
        in_features: int = 1280,
        classes: int = 4
    ):
        super(IsicSMPEffnetUnetModel, self).__init__()
        self.in_features = in_features
        
        self.encoder = get_encoder(
            encoder_name,
            in_channels=3,
            depth=5,
            weights=encoder_weights,
        )

        self.conv_head = self.encoder.conv_head
        self.bn2 = self.encoder.bn2
        self.global_pool = self.encoder.global_pool

        self.decoder = UnetDecoder(
            encoder_channels=self.encoder.out_channels,
            decoder_channels=decoder_channels,
            n_blocks=5,
            use_batchnorm=decoder_use_batchnorm,
            center=True if encoder_name.startswith("vgg") else False,
            attention_type=decoder_attention_type,
        )

        self.segmentation_head = SegmentationHead(
            in_channels=decoder_channels[-1],
            out_channels=1,
            activation=None,
            kernel_size=3,
        )
        self.fc = nn.Linear(in_features, 1024, bias=True)
        self.cls_head = nn.Linear(1024, classes, bias=True)

    @autocast()
    def forward(self, x):
        x = self.encoder(x)
        x = self.conv_head(x[-1])
        x = self.bn2(x)
        x = self.global_pool(x)
        x = x.view(-1, self.in_features)
        x = self.fc(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.cls_head(x)
        return x
#  модель Unet++ с предобученным энкодером Res2Next и улучшенным декодером для сегментации

class IsicSMPResnetUnetPlusPlusModel(nn.Module):
    def __init__(
        self,
        encoder_name = "timm-res2next50",
        encoder_weights = 'imagenet',
        decoder_use_batchnorm: bool = True,
        decoder_channels: List[int] = (256, 128, 64, 32, 16),
        decoder_attention_type: Optional[str] = None,
        in_features: int = 2048,
        classes: int = 4
    ):
        super(IsicSMPResnetUnetPlusPlusModel, self).__init__()
        self.in_features = in_features

        self.encoder = get_encoder(
            encoder_name,
            in_channels=3,
            depth=5,
            weights=encoder_weights,
        )

        self.global_pool = SelectAdaptivePool2d()

        self.decoder = UnetPlusPlusDecoder(
            encoder_channels=self.encoder.out_channels,
            decoder_channels=decoder_channels,
            n_blocks=5,
            use_batchnorm=decoder_use_batchnorm,
            center=True if encoder_name.startswith("vgg") else False,
            attention_type=decoder_attention_type,
        )

        self.segmentation_head = SegmentationHead(
            in_channels=decoder_channels[-1],
            out_channels=1,
            activation=None,
            kernel_size=3,
        )
        self.fc = nn.Linear(in_features, 1024, bias=True)
        self.cls_head = nn.Linear(1024, classes, bias=True)

    @autocast()
    def forward(self, x):
        x = self.encoder(x)
        x = self.global_pool(x[-1])
        x = y_cls.view(-1, self.in_features)
        x = self.fc(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.cls_head(x)
        return x

#  модель FPN с энкодером MIT и декодером Feature Pyramid Network для сегментации

class IsicSMPMitFPNModel(nn.Module):
    def __init__(
        self,
        encoder_name = "mit_b5",
        encoder_weights = 'imagenet',
        classes: int = 4
    ):
        super(IsicSMPMitFPNModel, self).__init__()
        self.in_features = 1280

        self.encoder = get_encoder(
            encoder_name,
            in_channels=3,
            depth=5,
            weights=encoder_weights,
        )
        
        if encoder_name == "mit_b5":
            self.conv_head = nn.Conv2d(512, self.in_features, 1, 1, bias=False)
        elif encoder_name == "mit_b0":
            self.conv_head = nn.Conv2d(256, self.in_features, 1, 1, bias=False)
        else:
            raise ValueError()
        self.bn2 = BatchNormAct2d(num_features=self.in_features)
        self.global_pool = SelectAdaptivePool2d()

        self.decoder = FPNDecoder(
            encoder_channels=self.encoder.out_channels,
            encoder_depth=5,
            pyramid_channels=256,
            segmentation_channels=128,
            dropout=0.2,
            merge_policy="add",
        )

        self.segmentation_head = SegmentationHead(
            in_channels=self.decoder.out_channels,
            out_channels=1,
            kernel_size=1,
            upsampling=4,
        )
        self.fc = nn.Linear(self.in_features, 1024, bias=True)
        self.cls_head = nn.Linear(1024, classes, bias=True)

    @autocast()
    def forward(self, x):
        x = self.encoder(x)

        x = self.conv_head(x[-1])
        x = self.bn2(x)
        x = self.global_pool(x)
        x = x.view(-1, self.in_features)
        x = self.fc(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.cls_head(x)
        return x
# Класс для объединения моделей классификации и сегментации с использованием различных энкодеров

class ISIC_Cls_Seg_Aux_Model(nn.Module):
    def __init__(self, encoder_name="timm-efficientnet-b0", encoder_weights = 'noisy-student', in_features=1280, classes=10, img_size=256):
        super(ISIC_Cls_Seg_Aux_Model, self).__init__()
        if "timm-efficientnet" in encoder_name:
            self.model = IsicSMPEffnetUnetModel(
                encoder_name=encoder_name,
                encoder_weights=encoder_weights,
                in_features=in_features,
                classes=classes)
        elif "timm-res2next50" in encoder_name:
            self.model = IsicSMPResnetUnetPlusPlusModel(
                encoder_name=encoder_name,
                encoder_weights=encoder_weights,
                in_features=in_features,
                classes=classes)
        elif "mit_b" in encoder_name:
            self.model = IsicSMPMitFPNModel(
                encoder_name=encoder_name,
                encoder_weights=encoder_weights,
                classes=classes)
        else:
            raise ValueError()
    @autocast()
    def forward(self, x):
        return self.model(x)
# класс датасета для загрузки изображений ISIC из HDF5 файла с применением различных трансформаций
class ISIC_Dataset(Dataset):
    # инициализация HDF5 файла и определение трансформаций для изображений разных размеров
    def __init__(self, df, file_hdf):
        self.fp_hdf = h5py.File(file_hdf, mode="r")
        self.isic_ids = df['isic_id'].values
        self.transform_64 = albu.Compose([
            albu.Resize(64, 64),
            albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ])
        self.transform_128 = albu.Compose([
            albu.Resize(128, 128),
            albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ])
        self.transform_224 = albu.Compose([
            albu.Resize(224, 224),
            albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ])
        self.transform_256 = albu.Compose([
            albu.Resize(256, 256),
            albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ])
        self.transform_288 = albu.Compose([
            albu.Resize(288, 288),
            albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ])
        self.transform_384 = albu.Compose([
            albu.Resize(384, 384),
            albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ])
    # возвращает количество изображений в датасете
    def __len__(self):
        return len(self.isic_ids)
    # загружает изображение по индексу, применяет трансформации и возвращает их в нескольких размерах
    def __getitem__(self, index):
        isic_id = self.isic_ids[index]
        image = np.array(Image.open(BytesIO(self.fp_hdf[isic_id][()])))
        image_64 = self.transform_64(image=image)['image']
        image_128 = self.transform_128(image=image)['image']
        image_224 = self.transform_224(image=image)['image']
        image_256 = self.transform_256(image=image)['image']
        image_288 = self.transform_288(image=image)['image']
        image_384 = self.transform_384(image=image)['image']
        return isic_id, image_64, image_128, image_224, image_256, image_288, image_384

In [5]:
df_sub = pd.read_csv("/kaggle/input/isic-2024-challenge/sample_submission.csv")
hdf5_test_file = '/kaggle/input/isic-2024-challenge/test-image.hdf5'
classes_2024 = ['target', 'MEL','BCC','SCC','NV']
USE_IMAGE_FEAT = True

In [6]:
if USE_IMAGE_FEAT:
    vit_tiny_384_model = ISIC_Model(model_name='vit_tiny_patch16_384.augreg_in21k_ft_in1k', pretrained=False, num_classes=len(classes_2024))
    vit_tiny_384_model.load_state_dict(torch.load('/kaggle/input/isic-2024-img-ckpts/img_ckpt_noval_exp3/vit_tiny_patch16_384.augreg_in21k_ft_in1k_384_epoch9_ema.pt'))
    vit_tiny_384_model.cuda()
    vit_tiny_384_model.eval()

    swin_tiny_256_model = ISIC_Model(model_name='swinv2_tiny_window8_256.ms_in1k', pretrained=False, num_classes=len(classes_2024))
    swin_tiny_256_model.load_state_dict(torch.load('/kaggle/input/isic-2024-img-ckpts/img_ckpt_noval_exp3/swinv2_tiny_window8_256.ms_in1k_256_epoch7_ema.pt'))
    swin_tiny_256_model.cuda()
    swin_tiny_256_model.eval()

    convnextv2_tiny_288_model = ISIC_Model(model_name='convnextv2_tiny.fcmae_ft_in22k_in1k', pretrained=False, num_classes=len(classes_2024))
    convnextv2_tiny_288_model.load_state_dict(torch.load('/kaggle/input/isic-2024-img-ckpts/img_ckpt_noval_exp3/convnextv2_tiny.fcmae_ft_in22k_in1k_288_epoch8_ema.pt'))
    convnextv2_tiny_288_model.cuda()
    convnextv2_tiny_288_model.eval()

    swin_tiny_224_model = ISIC_Model(model_name='swin_tiny_patch4_window7_224.ms_in1k', pretrained=False, num_classes=len(classes_2024))
    swin_tiny_224_model.load_state_dict(torch.load('/kaggle/input/isic-2024-img-ckpts/img_ckpt_noval_exp0/swin_tiny_patch4_window7_224.ms_in1k_224_epoch8_ema.pt'))
    swin_tiny_224_model.cuda()
    swin_tiny_224_model.eval()

    convnextv2_base_128_model = ISIC_Model(model_name='convnextv2_base.fcmae_ft_in22k_in1k', pretrained=False, num_classes=len(classes_2024))
    convnextv2_base_128_model.load_state_dict(torch.load('/kaggle/input/isic-2024-img-ckpts/img_ckpt_noval_exp0/convnextv2_base.fcmae_ft_in22k_in1k_128_epoch8_ema.pt'))
    convnextv2_base_128_model.cuda()
    convnextv2_base_128_model.eval()

    convnextv2_large_64_model = ISIC_Model(model_name='convnextv2_large.fcmae_ft_in22k_in1k', pretrained=False, num_classes=len(classes_2024))
    convnextv2_large_64_model.load_state_dict(torch.load('/kaggle/input/isic-2024-img-ckpts/img_ckpt_noval_exp0/convnextv2_large.fcmae_ft_in22k_in1k_64_epoch8_ema.pt'))
    convnextv2_large_64_model.cuda()
    convnextv2_large_64_model.eval()

    coatnet_224_model = ISIC_Model(model_name='coatnet_rmlp_1_rw_224.sw_in1k', pretrained=False, num_classes=len(classes_2024))
    coatnet_224_model.load_state_dict(torch.load('/kaggle/input/isic-2024-img-ckpts/img_ckpt_noval_exp0/coatnet_rmlp_1_rw_224.sw_in1k_224_epoch8_ema.pt'))
    coatnet_224_model.cuda()
    coatnet_224_model.eval()

    eb3_aux_224_model = ISIC_Cls_Seg_Aux_Model(encoder_name="timm-efficientnet-b3", encoder_weights=None, in_features=1536, classes=len(classes_2024))
    eb3_aux_224_model.load_state_dict(torch.load('/kaggle/input/isic-2024-img-ckpts/img_ckpt_noval_aux_exp1/timm-efficientnet-b3_224_epoch8_aux_ema.pt'))
    eb3_aux_224_model.cuda()
    eb3_aux_224_model.eval()

    mit_b0_aux_384_model = ISIC_Cls_Seg_Aux_Model(encoder_name="mit_b0", encoder_weights=None, in_features=1280, classes=len(classes_2024))
    mit_b0_aux_384_model.load_state_dict(torch.load('/kaggle/input/isic-2024-img-ckpts/img_ckpt_noval_aux_exp1/mit_b0_384_epoch8_aux_ema.pt'))
    mit_b0_aux_384_model.cuda()
    mit_b0_aux_384_model.eval()

    mit_b5_aux_224_model = ISIC_Cls_Seg_Aux_Model(encoder_name="mit_b5", encoder_weights=None, in_features=1280, classes=len(classes_2024))
    mit_b5_aux_224_model.load_state_dict(torch.load('/kaggle/input/isic-2024-img-ckpts/img_ckpt_noval_aux_exp1/mit_b5_224_epoch8_aux_ema.pt'))
    mit_b5_aux_224_model.cuda()
    mit_b5_aux_224_model.eval()

    test_dataset = ISIC_Dataset(df=df_sub, file_hdf=hdf5_test_file)
    test_loader = DataLoader(test_dataset, batch_size=64, num_workers=2, shuffle=False)
    
    test_isic_ids = []
    img_feats = []
    for batch_isic_id, batch_image_64, batch_image_128, batch_image_224, batch_image_256, batch_image_288, batch_image_384 in tqdm(test_loader):
        test_isic_ids.extend(batch_isic_id)
        batch_image_64 = batch_image_64.cuda()
        batch_image_128 = batch_image_128.cuda()
        batch_image_224 = batch_image_224.cuda()
        batch_image_256 = batch_image_256.cuda()
        batch_image_288 = batch_image_288.cuda()
        batch_image_384 = batch_image_384.cuda()

        batch_feat = []
        with torch.cuda.amp.autocast(), torch.no_grad():
            vit_tiny_384_pred = torch.sigmoid(vit_tiny_384_model(batch_image_384))
            vit_tiny_384_pred = vit_tiny_384_pred.data.cpu().numpy()[:,0]
            batch_feat.append(vit_tiny_384_pred)

            swin_tiny_256_pred = torch.sigmoid(swin_tiny_256_model(batch_image_256))
            swin_tiny_256_pred = swin_tiny_256_pred.data.cpu().numpy()[:,0]
            batch_feat.append(swin_tiny_256_pred)

            convnextv2_tiny_pred = torch.sigmoid(convnextv2_tiny_288_model(batch_image_288))
            convnextv2_tiny_pred = convnextv2_tiny_pred.data.cpu().numpy()[:,0]
            batch_feat.append(convnextv2_tiny_pred)

            swin_tiny_224_pred = torch.sigmoid(swin_tiny_224_model(batch_image_224))
            swin_tiny_224_pred = swin_tiny_224_pred.data.cpu().numpy()[:,0]
            batch_feat.append(swin_tiny_224_pred)

            convnextv2_base_128_pred = torch.sigmoid(convnextv2_base_128_model(batch_image_128))
            convnextv2_base_128_pred = convnextv2_base_128_pred.data.cpu().numpy()[:,0]
            batch_feat.append(convnextv2_base_128_pred)

            convnextv2_large_64_pred = torch.sigmoid(convnextv2_large_64_model(batch_image_64))
            convnextv2_large_64_pred = convnextv2_large_64_pred.data.cpu().numpy()[:,0]
            batch_feat.append(convnextv2_large_64_pred)

            coatnet_224_pred = torch.sigmoid(coatnet_224_model(batch_image_224))
            coatnet_224_pred = coatnet_224_pred.data.cpu().numpy()[:,0]
            batch_feat.append(coatnet_224_pred)

            eb3_aux_224_pred = torch.sigmoid(eb3_aux_224_model(batch_image_224))
            eb3_aux_224_pred = eb3_aux_224_pred.data.cpu().numpy()[:,0]
            batch_feat.append(eb3_aux_224_pred)

            mit_b5_aux_224_pred = torch.sigmoid(mit_b5_aux_224_model(batch_image_224))
            mit_b5_aux_224_pred = mit_b5_aux_224_pred.data.cpu().numpy()[:,0]
            batch_feat.append(mit_b5_aux_224_pred)

            mit_b0_aux_384_pred = torch.sigmoid(mit_b0_aux_384_model(batch_image_384))
            mit_b0_aux_384_pred = mit_b0_aux_384_pred.data.cpu().numpy()[:,0]
            batch_feat.append(mit_b0_aux_384_pred)

        batch_feat = np.stack(batch_feat, -1)
        img_feats.append(batch_feat)
    test_isic_ids = np.array(test_isic_ids)
    img_feats = np.concatenate(img_feats, axis=0)
    img_feats_dict = dict(zip(test_isic_ids, img_feats))

    del vit_tiny_384_model
    del swin_tiny_256_model
    del convnextv2_tiny_288_model
    del swin_tiny_224_model
    del convnextv2_base_128_model
    del convnextv2_large_64_model
    del coatnet_224_model
    del eb3_aux_224_model
    del mit_b0_aux_384_model
    del mit_b5_aux_224_model

    del test_isic_ids
    del img_feats

100%|██████████| 1/1 [00:01<00:00,  1.88s/it]


In [7]:
id_col = 'isic_id'
target_col = 'target'
group_col = 'patient_id'

# имеющиеся "с коробки" фичи и их описание
num_cols = [
    'age_approx',                        # Approximate age of patient at time of imaging.
    'clin_size_long_diam_mm',            # Maximum diameter of the lesion (mm).+
    'tbp_lv_A',                          # A inside  lesion.+
    'tbp_lv_Aext',                       # A outside lesion.+
    'tbp_lv_B',                          # B inside  lesion.+
    'tbp_lv_Bext',                       # B outside lesion.+ 
    'tbp_lv_C',                          # Chroma inside  lesion.+
    'tbp_lv_Cext',                       # Chroma outside lesion.+
    'tbp_lv_H',                          # Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).+
    'tbp_lv_Hext',                       # Hue outside lesion.+
    'tbp_lv_L',                          # L inside lesion.+
    'tbp_lv_Lext',                       # L outside lesion.+
    'tbp_lv_areaMM2',                    # Area of lesion (mm^2).+
    'tbp_lv_area_perim_ratio',           # Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.+
    'tbp_lv_color_std_mean',             # Color irregularity, calculated as the variance of colors within the lesion's boundary.
    'tbp_lv_deltaA',                     # Average A contrast (inside vs. outside lesion).+
    'tbp_lv_deltaB',                     # Average B contrast (inside vs. outside lesion).+
    'tbp_lv_deltaL',                     # Average L contrast (inside vs. outside lesion).+
    'tbp_lv_deltaLB',                    #
    'tbp_lv_deltaLBnorm',                # Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.+
    'tbp_lv_eccentricity',               # Eccentricity.+
    'tbp_lv_minorAxisMM',                # Smallest lesion diameter (mm).+
    'tbp_lv_nevi_confidence',            # Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.+,++
    'tbp_lv_norm_border',                # Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.+
    'tbp_lv_norm_color',                 # Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.+
    'tbp_lv_perimeterMM',                # Perimeter of lesion (mm).+
    'tbp_lv_radial_color_std_max',       # Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.+
    'tbp_lv_stdL',                       # Standard deviation of L inside  lesion.+
    'tbp_lv_stdLExt',                    # Standard deviation of L outside lesion.+
    'tbp_lv_symm_2axis',                 # Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.+
    'tbp_lv_symm_2axis_angle',           # Lesion border asymmetry angle.+
    'tbp_lv_x',                          # X-coordinate of the lesion on 3D TBP.+
    'tbp_lv_y',                          # Y-coordinate of the lesion on 3D TBP.+
    'tbp_lv_z',                          # Z-coordinate of the lesion on 3D TBP.+
]

# feature engineering на основе тех фич
new_num_cols = [
    'lesion_size_ratio',             # tbp_lv_minorAxisMM      / clin_size_long_diam_mm
    'lesion_shape_index',            # tbp_lv_areaMM2          / tbp_lv_perimeterMM **2
    'hue_contrast',                  # tbp_lv_H                - tbp_lv_Hext              abs
    'luminance_contrast',            # tbp_lv_L                - tbp_lv_Lext              abs
    'lesion_color_difference',       # tbp_lv_deltaA **2       + tbp_lv_deltaB **2 + tbp_lv_deltaL **2  sqrt  
    'border_complexity',             # tbp_lv_norm_border      + tbp_lv_symm_2axis
    'color_uniformity',              # tbp_lv_color_std_mean   / tbp_lv_radial_color_std_max

    'position_distance_3d',          # tbp_lv_x **2 + tbp_lv_y **2 + tbp_lv_z **2  sqrt
    'perimeter_to_area_ratio',       # tbp_lv_perimeterMM      / tbp_lv_areaMM2
    'area_to_perimeter_ratio',       # tbp_lv_areaMM2          / tbp_lv_perimeterMM
    'lesion_visibility_score',       # tbp_lv_deltaLBnorm      + tbp_lv_norm_color
    'symmetry_border_consistency',   # tbp_lv_symm_2axis       * tbp_lv_norm_border
    'consistency_symmetry_border',   # tbp_lv_symm_2axis       * tbp_lv_norm_border / (tbp_lv_symm_2axis + tbp_lv_norm_border)

    'color_consistency',             # tbp_lv_stdL             / tbp_lv_Lext
    'consistency_color',             # tbp_lv_stdL*tbp_lv_Lext / tbp_lv_stdL + tbp_lv_Lext
    'size_age_interaction',          # clin_size_long_diam_mm  * age_approx
    'hue_color_std_interaction',     # tbp_lv_H                * tbp_lv_color_std_mean
    'lesion_severity_index',         # tbp_lv_norm_border      + tbp_lv_norm_color + tbp_lv_eccentricity / 3
    'shape_complexity_index',        # border_complexity       + lesion_shape_index
    'color_contrast_index',          # tbp_lv_deltaA + tbp_lv_deltaB + tbp_lv_deltaL + tbp_lv_deltaLBnorm

    'log_lesion_area',               # tbp_lv_areaMM2          + 1  np.log
    'normalized_lesion_size',        # clin_size_long_diam_mm  / age_approx
    'mean_hue_difference',           # tbp_lv_H                + tbp_lv_Hext    / 2
    'std_dev_contrast',              # tbp_lv_deltaA **2 + tbp_lv_deltaB **2 + tbp_lv_deltaL **2   / 3  np.sqrt
    'color_shape_composite_index',   # tbp_lv_color_std_mean   + bp_lv_area_perim_ratio + tbp_lv_symm_2axis   / 3
    'lesion_orientation_3d',         # tbp_lv_y                , tbp_lv_x  np.arctan2
    'overall_color_difference',      # tbp_lv_deltaA           + tbp_lv_deltaB + tbp_lv_deltaL   / 3

    'symmetry_perimeter_interaction',# tbp_lv_symm_2axis       * tbp_lv_perimeterMM
    'comprehensive_lesion_index',    # tbp_lv_area_perim_ratio + tbp_lv_eccentricity + bp_lv_norm_color + tbp_lv_symm_2axis   / 4
    'color_variance_ratio',          # tbp_lv_color_std_mean   / tbp_lv_stdLExt
    'border_color_interaction',      # tbp_lv_norm_border      * tbp_lv_norm_color
    'border_color_interaction_2',
    'size_color_contrast_ratio',     # clin_size_long_diam_mm  / tbp_lv_deltaLBnorm
    'age_normalized_nevi_confidence',# tbp_lv_nevi_confidence  / age_approx
    'age_normalized_nevi_confidence_2',
    'color_asymmetry_index',         # tbp_lv_symm_2axis       * tbp_lv_radial_color_std_max

    'volume_approximation_3d',       # tbp_lv_areaMM2          * sqrt(tbp_lv_x**2 + tbp_lv_y**2 + tbp_lv_z**2)
    'color_range',                   # abs(tbp_lv_L - tbp_lv_Lext) + abs(tbp_lv_A - tbp_lv_Aext) + abs(tbp_lv_B - tbp_lv_Bext)
    'shape_color_consistency',       # tbp_lv_eccentricity     * tbp_lv_color_std_mean
    'border_length_ratio',           # tbp_lv_perimeterMM      / pi * sqrt(tbp_lv_areaMM2 / pi)
    'age_size_symmetry_index',       # age_approx              * clin_size_long_diam_mm * tbp_lv_symm_2axis
    'index_age_size_symmetry',       # age_approx              * tbp_lv_areaMM2 * tbp_lv_symm_2axis
]

cat_cols = ['sex', 'anatom_site_general', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple', 'attribution']
norm_cols = [f'{col}_patient_norm' for col in num_cols + new_num_cols]
special_cols = ['count_per_patient']
feature_cols = num_cols + new_num_cols + cat_cols + norm_cols + special_cols

# FE-функция
def read_data(path, err = 1e-5):
    return (
        pl.read_csv(path)
        .with_columns(
            pl.col('age_approx').cast(pl.String).replace('NA', np.nan).cast(pl.Float64),
        )
        .with_columns(
            pl.col(pl.Float64).fill_nan(pl.col(pl.Float64).median()), # You may want to impute test data with train
        )
        .with_columns(
            lesion_size_ratio              = pl.col('tbp_lv_minorAxisMM') / pl.col('clin_size_long_diam_mm'),
            lesion_shape_index             = pl.col('tbp_lv_areaMM2') / (pl.col('tbp_lv_perimeterMM') ** 2),
            hue_contrast                   = (pl.col('tbp_lv_H') - pl.col('tbp_lv_Hext')).abs(),
            luminance_contrast             = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs(),
            lesion_color_difference        = (pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2).sqrt(),
            border_complexity              = pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_symm_2axis'),
            color_uniformity               = pl.col('tbp_lv_color_std_mean') / (pl.col('tbp_lv_radial_color_std_max') + err),
        )
        .with_columns(
            position_distance_3d           = (pl.col('tbp_lv_x') ** 2 + pl.col('tbp_lv_y') ** 2 + pl.col('tbp_lv_z') ** 2).sqrt(),
            perimeter_to_area_ratio        = pl.col('tbp_lv_perimeterMM') / pl.col('tbp_lv_areaMM2'),
            area_to_perimeter_ratio        = pl.col('tbp_lv_areaMM2') / pl.col('tbp_lv_perimeterMM'),
            lesion_visibility_score        = pl.col('tbp_lv_deltaLBnorm') + pl.col('tbp_lv_norm_color'),
            combined_anatomical_site       = pl.col('anatom_site_general') + '_' + pl.col('tbp_lv_location'),
            symmetry_border_consistency    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border'),
            consistency_symmetry_border    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border') / (pl.col('tbp_lv_symm_2axis') + pl.col('tbp_lv_norm_border')),
        )
        .with_columns(
            color_consistency              = pl.col('tbp_lv_stdL') / pl.col('tbp_lv_Lext'),
            consistency_color              = pl.col('tbp_lv_stdL') * pl.col('tbp_lv_Lext') / (pl.col('tbp_lv_stdL') + pl.col('tbp_lv_Lext')),
            size_age_interaction           = pl.col('clin_size_long_diam_mm') * pl.col('age_approx'),
            hue_color_std_interaction      = pl.col('tbp_lv_H') * pl.col('tbp_lv_color_std_mean'),
            lesion_severity_index          = (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_eccentricity')) / 3,
            shape_complexity_index         = pl.col('border_complexity') + pl.col('lesion_shape_index'),
            color_contrast_index           = pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL') + pl.col('tbp_lv_deltaLBnorm'),
        )
        .with_columns(
            log_lesion_area                = (pl.col('tbp_lv_areaMM2') + 1).log(),
            normalized_lesion_size         = pl.col('clin_size_long_diam_mm') / pl.col('age_approx'),
            mean_hue_difference            = (pl.col('tbp_lv_H') + pl.col('tbp_lv_Hext')) / 2,
            std_dev_contrast               = ((pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2) / 3).sqrt(),
            color_shape_composite_index    = (pl.col('tbp_lv_color_std_mean') + pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_symm_2axis')) / 3,
            lesion_orientation_3d          = pl.arctan2(pl.col('tbp_lv_y'), pl.col('tbp_lv_x')),
            overall_color_difference       = (pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL')) / 3,
        )
        .with_columns(
            symmetry_perimeter_interaction = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_perimeterMM'),
            comprehensive_lesion_index     = (pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_eccentricity') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_symm_2axis')) / 4,
            color_variance_ratio           = pl.col('tbp_lv_color_std_mean') / pl.col('tbp_lv_stdLExt'),
            border_color_interaction       = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color'),
            border_color_interaction_2     = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color') / (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color')),
            size_color_contrast_ratio      = pl.col('clin_size_long_diam_mm') / pl.col('tbp_lv_deltaLBnorm'),
            age_normalized_nevi_confidence = pl.col('tbp_lv_nevi_confidence') / pl.col('age_approx'),
            age_normalized_nevi_confidence_2 = (pl.col('clin_size_long_diam_mm')**2 + pl.col('age_approx')**2).sqrt(),
            color_asymmetry_index          = pl.col('tbp_lv_radial_color_std_max') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            volume_approximation_3d        = pl.col('tbp_lv_areaMM2') * (pl.col('tbp_lv_x')**2 + pl.col('tbp_lv_y')**2 + pl.col('tbp_lv_z')**2).sqrt(),
            color_range                    = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs() + (pl.col('tbp_lv_A') - pl.col('tbp_lv_Aext')).abs() + (pl.col('tbp_lv_B') - pl.col('tbp_lv_Bext')).abs(),
            shape_color_consistency        = pl.col('tbp_lv_eccentricity') * pl.col('tbp_lv_color_std_mean'),
            border_length_ratio            = pl.col('tbp_lv_perimeterMM') / (2 * np.pi * (pl.col('tbp_lv_areaMM2') / np.pi).sqrt()),
            age_size_symmetry_index        = pl.col('age_approx') * pl.col('clin_size_long_diam_mm') * pl.col('tbp_lv_symm_2axis'),
            index_age_size_symmetry        = pl.col('age_approx') * pl.col('tbp_lv_areaMM2') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            ((pl.col(col) - pl.col(col).mean().over('patient_id')) / (pl.col(col).std().over('patient_id') + err)).alias(f'{col}_patient_norm') for col in (num_cols + new_num_cols)
        )
        .with_columns(
            count_per_patient = pl.col('isic_id').count().over('patient_id'),
        )
        .with_columns(
            pl.col(cat_cols).cast(pl.Categorical),
        )
        .to_pandas()
        .set_index(id_col)
    )

#onehotencoding функция
def preprocess(df_train, df_test):
    global cat_cols
    
    encoder = OneHotEncoder(sparse_output=False, dtype=np.int32, handle_unknown='ignore')
    encoder.fit(df_train[cat_cols])
    
    new_cat_cols = [f'onehot_{i}' for i in range(len(encoder.get_feature_names_out()))]

    df_train[new_cat_cols] = encoder.transform(df_train[cat_cols])
    df_train[new_cat_cols] = df_train[new_cat_cols].astype('category')

    df_test[new_cat_cols] = encoder.transform(df_test[cat_cols])
    df_test[new_cat_cols] = df_test[new_cat_cols].astype('category')

    for col in cat_cols:
        feature_cols.remove(col)

    feature_cols.extend(new_cat_cols)
    cat_cols = new_cat_cols
    
    return df_train, df_test

In [8]:
df_train = read_data('/kaggle/input/isic-2024-challenge/train-metadata.csv')
df_test = read_data("/kaggle/input/isic-2024-challenge/test-metadata.csv")

df_train, df_test = preprocess(df_train, df_test)

test_isic_ids = df_test.index.values

x_test_tab = df_test[feature_cols]
x_test_tab_xgb = df_test[feature_cols].values.copy()
x_test_tab_xgb[x_test_tab_xgb == -np.inf] = 0
x_test_tab_xgb[x_test_tab_xgb == np.inf] = 0
x_test_tab_xgb = xgb.DMatrix(x_test_tab_xgb)

if USE_IMAGE_FEAT:
    image_pred_classes = [
        'vit_tiny_384',
        'swin_tiny_256',
        'convnextv2_tiny_288',
        'swin_tiny_224',
        'convnextv2_base_128',
        'convnextv2_large_64',
        'coatnet_224',
        'eb3_aux_224',
        'mit_b5_aux_224',
        'mit_b0_aux_384',
    ]

    meta = []
    for isic_id, row in df_test.iterrows():
        feat = img_feats_dict[isic_id]
        meta.append(feat)
    df_test[image_pred_classes] = np.array(meta, dtype=float)
    del img_feats_dict

    feature_cols_img_tab = image_pred_classes + feature_cols

    x_test_tab_img = df_test[feature_cols_img_tab]
    x_test_tab_img_xgb = df_test[feature_cols_img_tab].values.copy()
    x_test_tab_img_xgb[x_test_tab_img_xgb == -np.inf] = 0
    x_test_tab_img_xgb[x_test_tab_img_xgb == np.inf] = 0
    x_test_tab_img_xgb = xgb.DMatrix(x_test_tab_img_xgb)

In [9]:
tab_pred = []
for fold in range(5):
    lgb_ckpt_dir = '/kaggle/input/isic-2024-tab-ckpts/tab_only_v2_exp3/lgb/fold{}'.format(fold)
    cb_ckpt_dir = '/kaggle/input/isic-2024-tab-ckpts/tab_only_v2_exp3/cb/fold{}'.format(fold)
    xgb_ckpt_dir = '/kaggle/input/isic-2024-tab-ckpts/tab_only_v2_exp3/xgb/fold{}'.format(fold)
    
    for rdir, _, files in os.walk(lgb_ckpt_dir):
        assert len(files) == 6
        for file in files:
            if '.txt' not in file:
                continue
            ckpt_path = os.path.join(rdir, file)
            lgb_model = lgb.Booster(model_file=ckpt_path)
            lgb_y_pred = lgb_model.predict(x_test_tab.values)
            tab_pred.append(lgb_y_pred)
    
    for rdir, _, files in os.walk(cb_ckpt_dir):
        assert len(files) == 6
        for file in files:
            if '.cbm' not in file:
                continue
            ckpt_path = os.path.join(rdir, file)
            cb_model = cb.CatBoostClassifier()
            cb_model.load_model(ckpt_path, format='cbm')
            cb_y_pred = cb_model.predict_proba(x_test_tab)[:, 1]
            tab_pred.append(cb_y_pred)
            
    for rdir, _, files in os.walk(xgb_ckpt_dir):
        assert len(files) == 6
        for file in files:
            if '.json' not in file:
                continue
            ckpt_path = os.path.join(rdir, file)
            xgb_model = xgb.Booster()
            xgb_model.load_model(ckpt_path)
            xgb_y_pred = xgb_model.predict(x_test_tab_xgb, iteration_range=(0, xgb_model.best_iteration+1))
            tab_pred.append(xgb_y_pred)

tab_pred = np.array(tab_pred)
tab_pred = gmean(tab_pred, 0)

In [10]:
if USE_IMAGE_FEAT:
    tab_img_pred = []
    for fold in range(5):
        lgb_ckpt_dir = '/kaggle/input/isic-2024-tab-ckpts/tab_img_10models_v2_exp3/lgb/fold{}'.format(fold)
        cb_ckpt_dir = '/kaggle/input/isic-2024-tab-ckpts/tab_img_10models_v2_exp3/cb/fold{}'.format(fold)
        xgb_ckpt_dir = '/kaggle/input/isic-2024-tab-ckpts/tab_img_10models_v2_exp3/xgb/fold{}'.format(fold)

        for rdir, _, files in os.walk(lgb_ckpt_dir):
            assert len(files) == 6
            for file in files:
                if '.txt' not in file:
                    continue
                ckpt_path = os.path.join(rdir, file)
                lgb_model = lgb.Booster(model_file=ckpt_path)
                lgb_y_pred = lgb_model.predict(x_test_tab_img.values)
                tab_img_pred.append(lgb_y_pred)

        for rdir, _, files in os.walk(cb_ckpt_dir):
            assert len(files) == 6
            for file in files:
                if '.cbm' not in file:
                    continue
                ckpt_path = os.path.join(rdir, file)
                cb_model = cb.CatBoostClassifier()
                cb_model.load_model(ckpt_path, format='cbm')
                cb_y_pred = cb_model.predict_proba(x_test_tab_img)[:, 1]
                tab_img_pred.append(cb_y_pred)

        for rdir, _, files in os.walk(xgb_ckpt_dir):
            assert len(files) == 6
            for file in files:
                if '.json' not in file:
                    continue
                ckpt_path = os.path.join(rdir, file)
                xgb_model = xgb.Booster()
                xgb_model.load_model(ckpt_path)
                xgb_y_pred = xgb_model.predict(x_test_tab_img_xgb, iteration_range=(0, xgb_model.best_iteration+1))
                tab_img_pred.append(xgb_y_pred)
    tab_img_pred = np.array(tab_img_pred)
    tab_img_pred = gmean(tab_img_pred, 0)
    
    pred_ens = 0.2*tab_pred + 0.8*tab_img_pred
    
else:
    pred_ens = tab_pred

In [11]:
df_sub = pd.DataFrame()
df_sub['isic_id'] = test_isic_ids
df_sub["target"] = pred_ens
df_sub.to_csv("submission.csv", index=False)

In [12]:
df_sub.head()

,isic_id,target
0,ISIC_0015657,0.123083
1,ISIC_0015729,0.056275
2,ISIC_0015740,0.097015
